In [ ]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
# utrikes födda, Afrika, Asien, Okänd,Statslös
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101E/FodelselandArK'

data = {
  "query": [
    {
      "code": "Fodelseland",
      "selection": {
        "filter": "agg:LandISOAlfa2Varldsdelar2",
        "values": [
          "AFR",
          "ASI3",
          "OSPEC2",
          "ÖSL"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "agg:Ålder5år",
        "values": [
          "-4",
          "5-9",
          "10-14",
          "15-19",
          "20-24",
          "25-29",
          "30-34",
          "35-39",
          "40-44",
          "45-49",
          "50-54",
          "55-59",
          "60-64",
          "65-69",
          "70-74",
          "75-79",
          "80-84",
          "85-89",
          "90-94",
          "95-99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(url,json=data)
r.status_code

In [ ]:
json = r.json()
json['data'][0]

In [ ]:
#parse death data from json to df 

data_list = []

nr_records = (len(json['data']))

for rec in range(nr_records):

    data_list.append ((json['data'][rec]['key'][0],json['data'][rec]['key'][1],
                       json['data'][rec]['key'][2],json['data'][rec]['key'][3],json['data'][rec]['values'][0]))

In [ ]:
# death rate df
foreign_born = pd.DataFrame(data_list,columns=['continent','age_grp','gender','year','pop'])
foreign_born['continent'].unique()

In [ ]:
foreign_born['pop'] = foreign_born['pop'].astype(int)
foreign_born['year'] =  foreign_born['year'].astype(int)
foreign_born

In [ ]:
foreign_born = foreign_born.groupby(['year','gender']).sum()
foreign_born

In [ ]:
foreign_born = foreign_born.unstack()
cols = ['foreign_male','foreign_female']
foreign_born.columns = cols
foreign_born['foreign_tot'] = foreign_born['foreign_male'] + foreign_born['foreign_female']
foreign_born['foreign_male_factor'] = foreign_born['foreign_male'] / foreign_born['foreign_female']
foreign_born

In [ ]:
# total population
# fetch population data

url2 = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860'
data2 = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "agg:Ålder5år",
        "values": [
          "-4",
          "5-9",
          "10-14",
          "15-19",
          "20-24",
          "25-29",
          "30-34",
          "35-39",
          "40-44",
          "45-49",
          "50-54",
          "55-59",
          "60-64",
          "65-69",
          "70-74",
          "75-79",
          "80-84",
          "85-89",
          "90-94",
          "95-99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2000",
          "2001",
          "2002",
          "2003",
          "2004",
          "2005",
          "2006",
          "2007",
          "2008",
          "2009",
          "2010",
          "2011",
          "2012",
          "2013",
          "2014",
          "2015",
          "2016",
          "2017",
          "2018",
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r2 = requests.post(url2,json=data2)
print (r2.status_code)
json2 = r2.json()

In [ ]:
#parse population data

data_list2 = []

nr_records2 = (len(json2['data']))

for rec2 in range(nr_records2):

    data_list2.append ((json2['data'][rec2]['key'][0],
                        json2['data'][rec2]['key'][1],json2['data'][rec2]['key'][2],json2['data'][rec2]['values'][0]))


In [ ]:
# put pop.data into df2

population = pd.DataFrame(data_list2,columns=['age_grp','gender','year','pop'])
population['year'] = population['year'].astype(int)
population['pop'] = population['pop'].astype(int)
population

In [ ]:
population = population.groupby('year').sum()
population

In [ ]:
#foreign_born['total_pop'] = population['pop']
foreign_born

In [ ]:
foreign_born['total_pop'] = population['pop']
foreign_born['foreign_ratio'] = foreign_born['foreign_tot'] / foreign_born['total_pop']
foreign_born['growth_factor'] = foreign_born['foreign_ratio'] / foreign_born['foreign_ratio'].shift()
foreign_born['other'] = foreign_born['total_pop'] - foreign_born['foreign_tot']
foreign_born['other_ratio'] = foreign_born['other'] / foreign_born['total_pop']
foreign_born

In [ ]:
(foreign_born[['foreign_ratio','other_ratio']] * 100).plot(
    kind='bar',stacked=True,color=['k','yellow'],
    figsize=(18,12),title='Sweden percentage population from Africa,Asia,Unspecified,Stateless')

plt.ylabel('percent')
plt.yticks(range(0,111,5))
plt.ylim([0,100])
plt.legend(['percent population from Africa/Asia/Unspecified/Stateless','percent population other'])
plt.savefig('swe_population_read_and_weep.jpg',format='jpg',dpi=600)